In [ ]:
import tensorflow as tf
import keras
import numpy as np
import os
print("done importing")

done importing


In [ ]:
print("preprocessing corpora")
kafka = open('kafka.txt', 'rb').read()
text1 = kafka.decode(encoding='utf-8')
norwood = open('norwood.txt', 'rb').read()
text2 = norwood.decode(encoding='utf-8')
checkpoint_dir = 'checkpoints'
text = text1+text2 # call me naive but i think this is fine !!
text = text.replace("\n", "")
text = text.replace("\r", "")

preprocessing corpora


In [ ]:
print("creating char dicts")
vocab = sorted(set(text))
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

creating char dicts


In [ ]:
print("batching input sequences")
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int) 
seq_length = 100 # The max. length for single input
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

batching input sequences


In [ ]:
print("shuffling input sequences")

BUFFER_SIZE = 10000
BATCH_SIZE = 64
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

shuffling input sequences


In [ ]:
print("building model..")

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

model = build_model(
    vocab_size = len(vocab), # no. of unique characters
    embedding_dim=embedding_dim, # 256
    rnn_units=rnn_units, # 1024
    batch_size=BATCH_SIZE)  # 64 for the traning

model.summary()

building model..
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           23040     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 90)            92250     
Total params: 4,053,594
Trainable params: 4,053,594
Non-trainable params: 0
_________________________________________________________________


In [ ]:
print("compiling model with custom loss...")
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

compiling model with custom loss...


In [ ]:
print("creating checkpoint directory")

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

creating checkpoint directory


In [ ]:
print("training model")
EPOCHS = 60
history = model.fit(dataset, 
                    epochs=EPOCHS, 
                    callbacks=[checkpoint_callback])

training model
Epoch 1/60
 52/244 [=====>........................] - ETA: 26s - loss: 0.9297

KeyboardInterrupt: ignored

In [ ]:
tf.train.load_checkpoint('./checkpoints/ckpt_34')

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights('./checkpoints/ckpt_34')
model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (1, None, 256)            23040     
_________________________________________________________________
gru_4 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_4 (Dense)              (1, None, 90)             92250     
Total params: 4,053,594
Trainable params: 4,053,594
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def generate_text(model, num_generate, temperature, start_string):
  input_eval = [char2idx[s] for s in start_string] # string to numbers (vectorizing)
  input_eval = tf.expand_dims(input_eval, 0) # dimension expansion
  text_generated = [] # Empty string to store our results
  model.reset_states() # Clears the hidden states in the RNN

  for i in range(num_generate): #Run a loop for number of characters to generate
    predictions = model(input_eval) # prediction for single character
    predictions = tf.squeeze(predictions, 0) # remove the batch dimension

    # using a categorical distribution to predict the character returned by the model
    # higher temperature increases the probability of selecting a less likely character
    # lower --> more predictable
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # The predicted character as the next input to the model
    # along with the previous hidden state
    # So the model makes the next prediction based on the previous character
    input_eval = tf.expand_dims([predicted_id], 0) 
    # Also devectorize the number and add to the generated text
    text_generated.append(idx2char[predicted_id]) 

  return (start_string + ''.join(text_generated))

In [ ]:
generated_text = generate_text(
                    model, 
                    num_generate=200, 
                    temperature=0.5, 
                    start_string=u"She")
print(generated_text)

She stopped at a book and study the steering wheel a small stereo. At night I was trying to tell me what he said, with a match. Storm Trooper went on said the windowpane and watch the stone. He took a de


In [ ]:
model.save('rnn-model')

INFO:tensorflow:Assets written to: rnn-model/assets


INFO:tensorflow:Assets written to: rnn-model/assets


In [ ]:
reconstructed_model = keras.models.load_model("rnn-model")

In [ ]:
reconstructed_model.compile()
reconstructed_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (1, None, 256)            23040     
_________________________________________________________________
gru_4 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_4 (Dense)              (1, None, 90)             92250     
Total params: 4,053,594
Trainable params: 4,053,594
Non-trainable params: 0
_________________________________________________________________


In [ ]:
generated_text = generate_text(
                    reconstructed_model, 
                    num_generate=200, 
                    temperature=0.3, 
                    start_string=u"Crow")
print(generated_text)

Crow says. "That's not what I mean. It's hard to forget this kind of school where it was in a hollow of food.But that isn't saying that this was the first time in my life. There was nothing funny going on


In [ ]:
model.save_weights('rnn-weights')

In [ ]:
new_model = build_model(90, embedding_dim=256, rnn_units=256, batch_size=1)

In [ ]:
idx2char

array([' ', '!', '"', '#', '$', '&', "'", '(', ')', '*', ',', '-', '.',
       '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';',
       '<', '=', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I',
       'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V',
       'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g',
       'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't',
       'u', 'v', 'w', 'x', 'y', 'z', 'à', 'ç', 'è', 'é', '—', '�'],
      dtype='<U1')